<a href="https://colab.research.google.com/github/K-J-HYEON/good/blob/add-input-method/20_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-*- coding: utf-8 -*-

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

#이미지가 저장될 폴더가 없다면 만듭니다.
import os 
if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")

np.random.seed(3) #tensorflow에서 랜덤하게 값 추출
tf.random.set_seed(3)

#생성자 모델을 만듭니다.
generator = Sequential() #모델 이름을 generator로 정하고 Sequential()함수를 호출
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2))) # 128은 임의로 정한 노드의 수 
# input_dim은 랜덤 백터를 준비해 집어넣으라는 뜻이다. 적절한 숫자를 임의로 넣어주면 된다.(100차원 크기의 랜덤 벡터를 준비해 집어넣으라는 뜻이다.)
# ReLU와 거의 비슷한 형태를 갖습니다. 입력 값이 음수일 때 완만한 선형 함수를 그려줍니다.
# https://leestation.tistory.com/776참고
# ReLU 함수는 x≦0일 때 경사가 사라져버려 학습 과정이 불안해질 수 있는 문제가 있었지만 LReLU는 x≦0일 때도 학습이 진행되기 때문에 
# ReLU 함수보다 효과적인 활성화 함수라고 생각할 수 있지만 
# 실제로 사용하게 되면 효과가 있는 경우도 있고, 없는 경우도 있어 언제 효과가 나타나는지에 관해 아직 밝혀진 바가 없다.


generator.add(BatchNormalization()) #배치정규화 과정 : 입력 데이터의 평균이 0, 분산이 1이 되도록 재배치하는 것인데, 
# 값을 일정하게 재배치하는 역할을 한다.=> 층의 개수가 늘어나도 안정적인 학습을 진행할 수 있다.
generator.add(Reshape((7, 7, 128)))  #컨볼루션 레이어가 받아들일 수 있는 형태로 바꾸어 주는 코드
# Conv2D()함수의 input_shape부분에 들어갈 형태로 정해 준다.

generator.add(UpSampling2D()) # 14 * 14의 이미지로 바뀐다. 즉, 이미지의 가로, 세로 크기를 2배씩 늘려준다.(this layer를 지나면서 크기가 14 * 14가 되고)
generator.add(Conv2D(64, kernel_size=5, padding='same')) # 커널 사이즈는 5 * 5짜리 필터를 사용하겠다.
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2))) #p.284 참고 0보다 작을경우 0.2를 곱하라(GAN에서는 기존 랠루함수를 쓰면 학습이 불안정해 지는 경우가 많아서 리키랠루를쓴다) 
generator.add(UpSampling2D())#(this layer를 지나면서 크기가 28 * 28이 된다)

generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
#노드 1개 /5*5커널 만들고 /제로패딩 해준다음/ 탄젠트 함수를 쓴다.

#판별자 모델을 만듭니다.
discriminator = Sequential() #시퀀셜 함수 호출
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same")) #노드 수 64개  5*5커널이 쓰이고 
# stride는 커널 윈도를 몇 칸씩 이동시킬지를 정하는 옵션(여러 칸 움직이게 하는 이유는 => 가로, 세로 크기가 더 줄어 들어 새로운 특징을 추출해준다.(드롭아웃이나 풀링처럼 새로운 필터를 적용한 효과)
#input_shape (행, 열, 색상=3 or 흑백=1) #맨 처음 층에는 입력되는 값을 알려주어야 한다.

# filters: 몇 개의 다른 종류의 필터를 활용할 것인지를 나타냄. 출력 모양의 깊이(depth) 를 결정한다.
# kernel_size: 연산을 수행할 때 윈도우의 크기를 의미한다.
# strides: 연산을 수행할 때 윈도우가 가로 그리고 세로로 움직이면서 내적 연산을 수행하는데, 한 번에 얼마나 움직일지를 의미한다.


discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
#판별자는 진짜와 가짜만 구분하면 되기 때문에 생성자에서처럼 upsampling을 사용하지 않고 stride나 dropout을 통해서 차원을 줄여주는 기능을 적극적으로 사용하면서
#컨볼루션 신경망 본래의 목적을 달성하면 된다.

discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same")) #노드 수 128개  5*5커널이 쓰이고
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())#1차원으로 만들어주고 (가로 * 세로의 2차원으로 진행된 과정을 1차원으로 바꾸어 주는 flatten()함수와)
discriminator.add(Dense(1, activation='sigmoid')) # 마지막 활성화 함수로 sigmoid() 사용
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

discriminator.trainable = False #판별자가 진짜냐 가짜냐 판별만 하게끔
#(판별자는 가져 온 모델을 가지고 판별만 하고 판별자 자신이 학습되지 않게끔)




In [ ]:


#생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(100,))#랜덤한 100개의 벡터를 케라스의 Input() 함수에 집어넣어서 생성자에 입력할 ginput을 만든다.
dis_output = discriminator(generator(ginput)) # 생성자 모델 generator()에 ginput을 입력한다. 
#그 결과 출력되는 28 * 28크기의 이미지가 그대로 판별자 모델 discriminator()입력값으로 들어감
#판별자는 이 인풋값으로 참과 거짓을 판별하는데 그 결과를 dis_output이라 한다.

gan = Model(ginput, dis_output) #keras의 model함수 이용해 ginput값과 위에 dus_output값을 넣어서 gan이라는 이름의 새로운 모델 만듬.
gan.compile(loss='binary_crossentropy', optimizer='adam')#이진 로스 함수와 아담 옵티마이저를 사용해서 gen모델을 compile한다.(컴퓨터가 번역하게 끔)
gan.summary()


#신경망을 실행시키는 함수를 만듭니다. (지금까지의 모든과정을 실행시킴)
def gan_train(epoch, batch_size, saving_interval):#gan_train이란 함수를 통해 학습을 진행(saving_interval = 중간 과정을 저장할 때 몇 번마다 한 번씩 저장할지 정하는)
    # / / 어느 구간에 저장할 saving 타이밍

  # MNIST 데이터 불러오기(손글씨 데이터)

  (X_train, _), (_, _) = mnist.load_data()  # 앞서 불러온 적 있는 MNIST를 다시 이용합니다. 


 # 단, 테스트과정은 필요없고 이미지만 사용할 것이기 때문에 X_train만 불러왔습니다.
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')#가로 28, 세로 28 픽셀이고 흑백이므로 1을 설정

  X_train = (X_train - 127.5) / 127.5  

  # 픽셀값은 0에서 255사이의 값입니다. 이전에 255로 나누어 줄때는 
  # 이를 0~1사이의 값으로 바꾸었던 것인데, 
  # 여기서는 127.5를 빼준 뒤 127.5로 나누어 줌으로 인해 -1에서 1사이의 값으로 바뀌게 됩니다.
  # X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

  true = np.ones((batch_size, 1)) #모두 참(1)이라는 레이블 값을 가진 배열을 만든다.
  fake = np.zeros((batch_size, 1)) #모두 가짜(0)이라는 레이블 값을 가진 열을 batch_size 길이만큼 만든다.

  for i in range(epoch):
          # 실제 데이터를 판별자에 입력하는 부분입니다.
          idx = np.random.randint(0, X_train.shape[0], batch_size)#numpy 라이브러리 random() 함수를 사용해서 실제 이미지를 랜덤하게 선택해서 불러온다.
          #np.random.randint(a, b, c)는 a부터 b까지의 숫자 중 하나를 랜덤하게 선택해 가져오는 과정을 c번 반복하라는 뜻 0부터 X_train 개수 사이의 숫자를 배치사이즈 만큼 반복해서 가져오겠다.
          #batch_size는 한 번에 몇 개의 실제 이미지와 몇 개의 가상 이미지를 판별자에 넣을지 결정하는 변수 
          # train데이터의 전체 데이터 배치 사이즈 만큼 추출 하겠다. = index
          imgs = X_train[idx] #위에서 선택된 idx에 해당하는 이미지를 불러온다
          d_loss_real = discriminator.train_on_batch(imgs, true) #위에서 정한 true만큼 batch_size 길이만큼 만든다. 
          #판별자 모델에 train_on_batch() 함수를 써서 판별을 시작한다. 
          #ex)train_on_batch(x, y)함수는 입력 값(x)과 레이블(y)을 받아서 딱 한 번 학습을 실시해 모델을 업데이트 한다.
                #즉 x에는 imgs y에는 true를 넣고 준비를 마친다.

          # 배치사이즈를 트루로 인식하겠다 
        #이미지와 레이블  = imgs, true



          #가상 이미지를 판별자에 입력하는 부분입니다.
          #학습이 반복될수록 가짜라는 레이블을 붙인 이미지들에 대한 예측 결과가 거짓으로 나온다.
          noise = np.random.normal(0, 1, (batch_size, 100)) #생성자에 집어넣을 가상 이미지
          # 정수가 아니기 때문에 np.random.normal() 함수를 사용했는데
          # 여기서 (batch_size,100)은 batch_size만큼 100열을 뽑으라는 의미

          gen_imgs = generator.predict(noise)#noise가 생성자에 들어가고 결괏값이 gen_imgs로 저장된다.
          d_loss_fake = discriminator.train_on_batch(gen_imgs, fake) #gen_imgs에 fake(모두가짜(0))라는 레이블이 붙는다.
          # gen_imgs한테 fake라고 학습을 시키는 과정
            

          #판별자와 생성자의 오차를 계산합니다.
          d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) #d_loss_real과 d_loss_fake를 더해서 둘로 나눈 평균이 판별차의 오차

          g_loss = gan.train_on_batch(noise, true) #판별자와 생성자를 연결해서 만든 gan 모델을 이용해 생성자의 오차, g_loss를 구한다.
          #train_on_batch() 함수와 앞서 만든 gen_imgs를 사용한다. 생성자의 레이블은 무조건 참(1) 해놓고 판별자로 넘긴다.

          print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss) #학습이 진행되는 동안 생성자와 판별자의 오차가 출력된다.


        # 이부분은 중간 과정을 이미지로 저장해 주는 부분입니다. 본 장의 주요 내용과 관련이 없어
        # 소스코드만 첨부합니다. 만들어진 이미지들은 gan_images 폴더에 저장됩니다.
          if i % saving_interval == 0:
              #r, c = 5, 5
              noise = np.random.normal(0, 1, (25, 100)) #0, 1 사이에서 25~100까지
              gen_imgs = generator.predict(noise)

              # Rescale images 0 - 1
              gen_imgs = 0.5 * gen_imgs + 0.5

              fig, axs = plt.subplots(5, 5)
              count = 0
              for j in range(5):
                  for k in range(5):
                      axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                      axs[j, k].axis('off')
                      count += 1
              fig.savefig("gan_images/gan_mnist_%d.png" % i)

gan_train(4001, 32, 200)  #4000번 반복되고(+1을 해 주는 것에 주의), 배치 사이즈는 32,  200번 마다 결과가 저장되게 하였습니다.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_3 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________
11493376/11490434 [==============================] - 0s 0us/step
epoch:0  d_loss:0.7053  g_loss:0.6912
epoch:1  d_loss:0.4617  g_loss:0.3278
epoch:2  d_loss:0.5702  g_loss:0.1026
epoch:3  d_loss:0.6598  g_loss:0.0842
epoch:4  d_loss:0.5770  g_loss:0.1651
epoch:5  d_loss:0.5089  g_loss:0.4225
epoch:6  d_loss:0.4875  g_loss:0.6988
epoch:7  d_loss:0.5093  g_loss:0.8724
epoch:8 

In [ ]:
np.zeros((batch_size , 0))